# Launch jobs
Code to create a batch script for launching jobs on cori \
Sep 1, 2020

In [1]:
import os
import glob
import time
import subprocess as sp
import numpy as np

In [2]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/run_scripts


In [3]:
img_size=128
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/',
         '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/512_sqr/'}

results_loc='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size)

In [4]:
# ls /global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20200901_200005_bsize50_30eph/chkpt/trainer0/sgd.shared.training.epoch.30.step.13500/model0/

In [5]:
# glob.glob(results_loc+'*')

In [6]:
# [(8, 7200), (10, 9570), (12, 10950), (13, 12240), (15, 13390), (15, 13540)]
# (11,10290), (12,10680), (13,11980), (14,12460) (19,17570)

In [7]:
### Default dictionary
dict_pars={'epochs':2, 'seed':282, 'nodes':1, 'procs':8,'job_name':'gen_img',
#            'spec_loss':'',
           'spec_loss':'--spec_loss','mcr':'',
           'lambda_spec':0.5,'learn_rate':0.002,
          'time':'00:15:00', 'batchsize':550, 'step_interval':1}

fldr,epoch,step='20210226_175209_bsize256_scale0.1_deterministic_off',14,12460
data_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{3}square/{0}/chkpt/trainer0/sgd.shared.training_batch_begin.epoch.{1}.step.{2}/model0/'.format(fldr,epoch,step,img_size)

dict_pars['data_dir']=data_dir
dict_pars['code_dir']=code_dirs[str(img_size)]
print(dict_pars)


{'epochs': 2, 'seed': 282, 'nodes': 1, 'procs': 8, 'job_name': 'gen_img', 'spec_loss': '--spec_loss', 'mcr': '', 'lambda_spec': 0.5, 'learn_rate': 0.002, 'time': '00:15:00', 'batchsize': 550, 'step_interval': 1, 'data_dir': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210226_175209_bsize256_scale0.1_deterministic_off/chkpt/trainer0/sgd.shared.training_batch_begin.epoch.14.step.12460/model0/', 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/'}


In [8]:
assert os.path.isdir(dict_pars['data_dir'])

In [9]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

### Initial setup
module purge
module load cgpu
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python/3.7-anaconda-2019.10
module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install_20210223/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
$SPACK_ROOT/share/spack/setup-env.sh

export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir={code_dir}
data_dir={data_dir}                                                 

python $code_dir/test_exagan.py --seed {seed} -b {batchsize} -stp {step_interval} {mcr} {spec_loss} --lambda_spec {lambda_spec} -lr {learn_rate} --nodes {nodes} --procs {procs} --suffix {job_name} --epochs {epochs} -dr $data_dir 
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [10]:
# bash_strg

In [11]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'
fname='batch_gen_images.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/batch_gen_images.sh


In [12]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8
#SBATCH --gpus-per-task=1
#SBATCH --time=00:15:00
#SBATCH --job-name=gen_img

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

### Initial setup
module purge
module load cgpu
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python/3.7-anaconda-2019.10
module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install_20210223/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
$SPACK_ROOT/share/spack/setup-env.sh

export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/
data_dir=/global/cscratch1/sd/vpa/proj/cosmogan/re

In [13]:
### Move to staging locations in project space:
os.chdir(staging_loc)

### Submit jobs to cori queue

In [14]:
%%bash -s "$filename" ## Use python variable in bash
module load esslurm
sbatch $1

Submitted batch job 1590609
